# **PROYECTO 1 BI - DETECCIÓN DE FAKE NEWS**
---

Integrantes equipo #11:
*   Estudiante #1: Juan Pablo Barón - 202210502
*   Estudiante #2: María José Amorocho - 202220179
*   Estudiante #3: Julian Mondragón - 202221122

---

# 0. Carga de datos

In [90]:
from pandas import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re, string, unicodedata
from num2words import num2words
from nltk.corpus import stopwords
import spacy


# Punkt permite separar un texto en frases.
#nltk.download('punkt')
#Descargar palabras vacías (stopwords)
#nltk.download("stopwords")
stop_words = set(stopwords.words("spanish"))


data_set = pd.read_csv('./Data/fake_news_spanish.csv', delimiter=";")
data_set_augmented = pd.read_csv('./Data/Noticias_Falsas_Extendidas_50-50.csv', delimiter=",")
data_set_augmented_copy = data_set_augmented.copy()

data_set_augmented.head()

,ID,Label,Titulo,Descripcion,Fecha
0,ID,1,'The Guardian' va con Sánchez: 'Europa necesit...,El diario británico publicó este pasado jueves...,02/06/2023
1,ID,0,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,REVELAN QUE EL GOBIERNO NEGOCIO LA LIBERACIÓN ...,01/10/2023
2,ID,1,El 'Ahora o nunca' de Joan Fuster sobre el est...,El valencianismo convoca en Castelló su fiesta...,25/04/2022
3,ID,1,"Iglesias alienta a Yolanda Díaz, ERC y EH Bild...","En política, igual que hay que negociar con lo...",03/01/2022
4,ID,0,Puigdemont: 'No sería ninguna tragedia una rep...,"En una entrevista en El Punt Avui, el líder de...",09/03/2018


---

 # **2. Entendimiento y preparación de los datos**

### 2.1 Selección de variables

In [91]:
def definir_variables(data_set_inicial):
    features = ['Titulo', 'Descripcion', 'Label']
    return data_set_inicial[features]


### 2.3 Preparación de datos

#### 2.3.1 Limpieza de datos

In [92]:
def remove_duplicates(df: pd.DataFrame):
    """
    Elimina las filas duplicadas del dataSet.
    
    Parámetros:
    df (pd.DataFrame): DataFrame de entrada.
    
    Retorna:
    pd.DataFrame: DataFrame sin filas duplicadas.
    """
    return df.drop_duplicates().reset_index(drop=True)


In [93]:
def eliminar_duplicados_parciales(df):
    """
    Elimina filas donde 'Titulo' y 'Descripcion' sean iguales, pero 'Label' sea diferente.
    
    Parámetros:
    df (pd.DataFrame): DataFrame con las columnas 'Titulo', 'Descripcion' y 'Label'.
    
    Retorna:
    pd.DataFrame: DataFrame sin los duplicados parciales.
    """
    # Contar cuántos valores únicos de Label existen por cada combinación de Titulo y Descripcion
    conteo_labels = df.groupby(['Titulo', 'Descripcion'])['Label'].nunique()
    
    # Identificar las combinaciones que tienen más de un Label distinto (es decir, duplicados parciales)
    duplicados_parciales = conteo_labels[conteo_labels > 1].index
    
    # Filtrar el DataFrame eliminando estas combinaciones
    df_filtrado = df[~df.set_index(['Titulo', 'Descripcion']).index.isin(duplicados_parciales)]
    
    return df_filtrado.reset_index(drop=True)



In [94]:
def limpiar_data(data_set):
    df_variables = definir_variables(data_set)
    df_duplicados = remove_duplicates(df_variables)
    df_limpio = eliminar_duplicados_parciales(df_duplicados)
    return df_limpio


#### 2.3.3 Tokenización, lematización y normalización

In [95]:
import swifter

# Cargar modelo spaCy sin parser ni NER para mayor velocidad
nlp = spacy.load("es_core_news_sm", disable=["parser", "ner"])
stop_words = set(stopwords.words("spanish"))

def preprocessing2(words):
    words = [word.lower() for word in words]  # Convertir a minúsculas directamente
    words = [num2words(word, lang="es") if word.isdigit() else word for word in words]  # Reemplazar números
    words = [re.sub(r'[^\w\s]', '', word) for word in words if word]  # Eliminar puntuación
    words = [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words]  # Quitar caracteres no ASCII
    words = [word for word in words if word not in stop_words]  # Remover stopwords
    words = [word for word in words if word.strip() != ""] 
    return words

def procesar_texto(df, columnas):
    """
    Optimiza la tokenización, lematización y preprocesamiento de múltiples columnas de un DataFrame.
    """
    df = df.copy()
    df[columnas] = df[columnas].astype(str).fillna("")
    
    datos_procesados = {}
    
    for col in columnas:
        resultados = [
            ([token.text for token in doc if not token.is_space],  # Tokens
             [token.lemma_ for token in doc if not token.is_space])  # Lemas
            for doc in nlp.pipe(df[col], batch_size=100)  # Mayor batch_size para eficiencia
        ]
        
        # Separar tokens y lemas en listas
        df[f"{col}_tokens"], df[f"{col}_lemmas"] = zip(*resultados)
        
        # Aplicar procesamiento en paralelo
        datos_procesados[f"{col}_tokens_clean"] = df[f"{col}_tokens"].swifter.apply(preprocessing2)
    
    # Crear nuevo DataFrame solo con las columnas procesadas + label
    nuevo_df = pd.DataFrame(datos_procesados)
    nuevo_df.insert(0, "Label", df["Label"])  # Mantener la etiqueta
    
    return nuevo_df


In [101]:
def limpiar_y_procesar(data_set, columnas_texto):
    """
    Función que aplica limpieza de datos y tokenización en columnas de texto específicas.

    Parámetros:
    - data_set (DataFrame): Dataset original con las columnas a procesar.
    - columnas_texto (list): Lista de nombres de columnas que contienen texto a limpiar.

    Retorna:
    - DataFrame con los textos procesados.
    """
    # Aplicar limpieza de datos
    data_set_limpio = limpiar_data(data_set)
    print(" Data limpia")

    # Aplicar tokenización, lematización y normalización
    df_procesado = procesar_texto(data_set_limpio, columnas_texto)
    
    return df_procesado

# Uso de la función
# df_procesado = limpiar_y_procesar(data_set, ["Titulo", "Descripcion"])
df_procesado = limpiar_y_procesar(data_set_augmented, ["Titulo", "Descripcion"])
df_procesado.head(10)

# df = limpiar_y_procesar(data_set, ["Titulo", "Descripcion"])

 Data limpia


Pandas Apply: 100%|██████████| 64062/64062 [00:26<00:00, 2440.29it/s]


,Label,Titulo_tokens_clean,Descripcion_tokens_clean
0,1,"[the, guardian, va, sanchez, europa, necesita,...","[diario, britanico, publico, pasado, jueves, e..."
1,0,"[revelan, gobierno, negocio, liberacion, mirel...","[revelan, gobierno, negocio, liberacion, mirel..."
2,1,"[ahora, nunca, joan, fuster, estatuto, valenci...","[valencianismo, convoca, castello, fiesta, gra..."
3,1,"[iglesias, alienta, yolanda, diaz, erc, eh, bi...","[politica, igual, negociar, empresarios, negoc..."
4,0,"[puigdemont, seria, ninguna, tragedia, repetic...","[entrevista, punt, avui, lider, jxcat, desdram..."
5,1,"[pnv, consolida, mayoria, pse, salva, papeles,...","[nacionalistas, consiguen, alcaldias, bilbao, ..."
6,0,"[exconsejero, nuria, marin, pide, indulto, cas...","[familiares, aluden, honestidad, integridad, p..."
7,1,"[fiscalia, pide, prision, incondicional, siete...","[suprime, delito, rebelion, imputo, inicialmen..."
8,1,"[jose, manuel, perez, tornero, creador, televi...","[futuro, presidente, rtve, licenciado, ciencia..."
9,0,"[ayusizacion, bng, santiago, abascal, instruye...","[pablo, santiago, abascal, planea, vivir, rent..."


#### 2.3.4 Vectorización

In [102]:
# Convertir listas de tokens a strings correctamente
df_procesado["Titulo_tokens_clean"] = df_procesado["Titulo_tokens_clean"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
df_procesado["Descripcion_tokens_clean"] = df_procesado["Descripcion_tokens_clean"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

print(df_procesado[["Titulo_tokens_clean", "Descripcion_tokens_clean"]].head())


                                 Titulo_tokens_clean  \
0  the guardian va sanchez europa necesita apuest...   
1  revelan gobierno negocio liberacion mireles ca...   
2  ahora nunca joan fuster estatuto valenciano cu...   
3  iglesias alienta yolanda diaz erc eh bildu neg...   
4  puigdemont seria ninguna tragedia repeticion e...   

                            Descripcion_tokens_clean  
0  diario britanico publico pasado jueves editori...  
1  revelan gobierno negocio liberacion mireles ca...  
2  valencianismo convoca castello fiesta grande c...  
3  politica igual negociar empresarios negociar g...  
4  entrevista punt avui lider jxcat desdramatizad...  


### **Clase pipeline limpieza y preprocesamiento**

In [98]:
from sklearn.base import BaseEstimator, TransformerMixin

class LimpiezaPreprocesamiento1(BaseEstimator, TransformerMixin):
    def __init__(self):
        from nltk.corpus import stopwords
        self.stop_words = set(stopwords.words("spanish"))

    def definir_variables(self, df):
        features = ['Titulo', 'Descripcion', 'Label']
        return df[features] if "Label" in df.columns else df[["Titulo", "Descripcion"]]

    def remove_duplicates(self, df):
        return df.drop_duplicates().reset_index(drop=True)

    def eliminar_duplicados_parciales(self, df):
        if "Label" not in df.columns:
            return df
        conteo_labels = df.groupby(['Titulo', 'Descripcion'])['Label'].nunique()
        duplicados_parciales = conteo_labels[conteo_labels > 1].index
        return df[~df.set_index(['Titulo', 'Descripcion']).index.isin(duplicados_parciales)].reset_index(drop=True)

    def limpiar_data(self, df):
        df_variables = self.definir_variables(df)
        df_duplicados = self.remove_duplicates(df_variables)
        return self.eliminar_duplicados_parciales(df_duplicados)

    def preprocessing(self, words):
        import re, unicodedata
        from num2words import num2words

        words = [word.lower() for word in words]
        words = [num2words(word, lang="es") if word.isdigit() else word for word in words]
        words = [re.sub(r'[^\w\s]', '', word) for word in words if word]
        words = [unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore') for word in words]
        words = [word for word in words if word not in self.stop_words]
        return [word for word in words if word.strip() != ""]

    def transform(self, X, y=None):
        import pandas as pd
        import spacy

        df = X.copy()
        df = self.limpiar_data(df)

        df["Titulo"] = df["Titulo"].astype(str).fillna("")
        df["Descripcion"] = df["Descripcion"].astype(str).fillna("")

        nlp = spacy.load("es_core_news_sm", disable=["parser", "ner"])

        resultados_titulo = [
            [token.text for token in doc if not token.is_space]
            for doc in nlp.pipe(df["Titulo"], batch_size=100)
        ]
        resultados_descripcion = [
            [token.text for token in doc if not token.is_space]
            for doc in nlp.pipe(df["Descripcion"], batch_size=100)
        ]

        df["Titulo_tokens_clean"] = [self.preprocessing(t) for t in resultados_titulo]
        df["Descripcion_tokens_clean"] = [self.preprocessing(d) for d in resultados_descripcion]

        df["Titulo_tokens_clean"] = df["Titulo_tokens_clean"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)
        df["Descripcion_tokens_clean"] = df["Descripcion_tokens_clean"].apply(lambda x: " ".join(x) if isinstance(x, list) else x)

        return df[["Titulo_tokens_clean", "Descripcion_tokens_clean"]]

    def fit(self, X, y=None):
        return self


In [103]:
# Definir la cantidad de muestras por clase
#df_procesado = data_set_augmented_copy
n_samples = 8500 #LO MOVI

# Extraer 8500 noticias falsas y 8500 noticias verídicas para pruebas finales
df_test_uniform = (df_procesado.groupby("Label", group_keys=False)
                    .apply(lambda x: x.sample(n=n_samples, random_state=42)))

# Crear el nuevo dataset de entrenamiento sin los registros seleccionados para pruebas
df_train = df_procesado.drop(df_test_uniform.index)

# Verificar tamaños de los datasets
print(f"✅ Dataset de entrenamiento: {len(df_train)} registros")
print(f"✅ Dataset de prueba uniforme (17k registros): {len(df_test_uniform)} registros (8500 fake news y 8500 verídicas)")


✅ Dataset de entrenamiento: 47062 registros
✅ Dataset de prueba uniforme (17k registros): 17000 registros (8500 fake news y 8500 verídicas)


C:\Users\majoa\AppData\Local\Temp\ipykernel_7416\5412614.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=n_samples, random_state=42)))


Para la vectorización de textos se hace uso de la librería TF-IDF (Term Frequency-Inverse Document Frequency). 

In [104]:
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=3000)
X_titulo = vectorizer.fit_transform(df_procesado["Titulo_tokens_clean"])
X_descripcion = vectorizer.fit_transform(df_procesado["Descripcion_tokens_clean"])

#  Concatenar ambas representaciones en una sola matriz
X = hstack([X_titulo, X_descripcion])

# Etiquetas
y = df_procesado["Label"]

# Verificar dimensiones de la matriz X y etiquetas y
print(f"Número de registros en X: {X.shape[0]}")
print(f"Número de etiquetas en y: {y.shape[0]}")

# Comprobar si hay inconsistencia
if X.shape[0] == y.shape[0]:
    print(" X y y tienen el mismo número de registros. OK.")
else:
    print(" Error: X y y tienen un número diferente de registros.")

Número de registros en X: 64062
Número de etiquetas en y: 64062
 X y y tienen el mismo número de registros. OK.


---

# **3. Modelado y evaluación**

### 3.3. Estudiante 3 (Julian Mondragón): Modelo 3


In [105]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time
from limpieza_transformer import LimpiezaPreprocesamiento
import joblib

# Medir tiempo de ejecución
start_time = time.time()

# SEPARAR el preprocesamiento de la vectorización 
tfidf_transformer = ColumnTransformer(
    transformers=[
        ('tfidf_titulo', TfidfVectorizer(ngram_range=(1,3), max_features=6000), "Titulo_tokens_clean"),
        ('tfidf_descripcion', TfidfVectorizer(ngram_range=(1,3), max_features=6000), "Descripcion_tokens_clean")
    ]
)

# Crear pipeline donde el vectorizador **solo se ajusta dentro de cada fold**
modelo_rf = Pipeline([
    ('limpieza', LimpiezaPreprocesamiento()),
    ('vectorizacion', tfidf_transformer),
    ('clasificador', RandomForestClassifier(n_estimators=100, random_state=42))  
])

# Definir K-Folds estratificado (10 folds)
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Cross-validation donde `TfidfVectorizer` se ajusta dentro de cada fold
print(" Ejecutando Cross-Validation con Random Forest usando df_train...")

y_pred_cv = cross_val_predict(modelo_rf, df_train, df_train["Label"], cv=kf, n_jobs=1, verbose=1)

# Calcular métricas en cross-validation
accuracy_cv = accuracy_score(df_train["Label"], y_pred_cv)
precision_cv = precision_score(df_train["Label"], y_pred_cv, average='binary')
recall_cv = recall_score(df_train["Label"], y_pred_cv, average='binary')
f1_cv = f1_score(df_train["Label"], y_pred_cv, average='binary')
conf_matrix_cv = confusion_matrix(df_train["Label"], y_pred_cv)

# Mostrar resultados de validación cruzada
print("\n Resultados de Cross-Validation con Random Forest en df_train:")
print(f" Accuracy: {accuracy_cv:.4f}")
print(f" Precision: {precision_cv:.4f}")
print(f" Recall: {recall_cv:.4f}")
print(f" F1-Score: {f1_cv:.4f}")
print("\n Matriz de Confusión:")
print(conf_matrix_cv)

# Reporte de clasificación
print("\n Reporte de Clasificación en Cross-Validation:")
print(classification_report(df_train["Label"], y_pred_cv))

# Medir tiempo total de entrenamiento
end_time = time.time()
print(f"\n Tiempo total de entrenamiento: {(end_time - start_time) / 60:.2f} minutos")

# -------------------------------------------
# **ENTRENAMIENTO FINAL con df_train y evaluación en df_test_uniform**
# -------------------------------------------

# Ahora entrenamos el modelo en todo df_train (sin test leakage)
modelo_rf.fit(df_train, df_train["Label"])
# Guardar el pipeline en un archivo
joblib.dump(modelo_rf, "modelo_random_forest.pkl")

# Transformamos df_test_uniform usando los valores aprendidos en df_train
y_pred_test = modelo_rf.predict(df_test_uniform)

# Calcular métricas en prueba final
accuracy_test = accuracy_score(df_test_uniform["Label"], y_pred_test)
precision_test = precision_score(df_test_uniform["Label"], y_pred_test, average='binary')
recall_test = recall_score(df_test_uniform["Label"], y_pred_test, average='binary')
f1_test = f1_score(df_test_uniform["Label"], y_pred_test, average='binary')
conf_matrix_test = confusion_matrix(df_test_uniform["Label"], y_pred_test)

# Mostrar resultados en el conjunto de prueba final
print("\nResultados en el **Conjunto de Prueba Final (df_test_uniform)**:")
print(f" Accuracy: {accuracy_test:.4f}")
print(f" Precision: {precision_test:.4f}")
print(f" Recall: {recall_test:.4f}")
print(f" F1-Score: {f1_test:.4f}")
print("\n Matriz de Confusión:")
print(conf_matrix_test)

# Reporte de clasificación en prueba final
print("\n Reporte de Clasificación en df_test_uniform:")
print(classification_report(df_test_uniform["Label"], y_pred_test))


 Ejecutando Cross-Validation con Random Forest usando df_train...


KeyError: "['Titulo', 'Descripcion'] not in index"